<a href="https://colab.research.google.com/github/sag005/spotify-playlist-recommendation/blob/main/Popularity_Based_Reco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pickle as pkl
from collections import defaultdict
import pandas as pd
import random
import numpy as np
from scipy.spatial import distance
from sklearn.metrics import ndcg_score
import math
import warnings

In [2]:
tracks_data = pd.read_pickle('track_data.pkl')
playlists_data = pd.read_pickle('playlist_data.pkl')

In [3]:
artist_tracks_dict = {}
with open('artist_track_mapping.pkl', 'rb') as handle:
    artist_tracks_dict = pkl.load(handle)

In [4]:
track_artist_mapping = {}
with open('track_artist_mapping.pkl', 'rb') as handle:
    track_artist_mapping = pkl.load(handle) 

In [5]:
track_frequency = {}
with open('track_frequency.pkl', 'rb') as handle:
    track_frequency = pkl.load(handle)

In [6]:
artist_tracks_count_dict = {}
with open('artist_track_count.pkl', 'rb') as handle:
    artist_tracks_count_dict = pkl.load(handle)

In [7]:
def getArtistsForTracks(tracks):
  return [track_artist_mapping[track] for track in tracks]

In [8]:
def R_precision(predicted_tracks, with_held_tracks):
  # print(len(set(with_held_tracks)))
  intersection = len(set(predicted_tracks).intersection(set(with_held_tracks)))
  return intersection / len(set(with_held_tracks))

In [73]:
import copy
def NDCG(truth, pred, K):
  pred_copy = copy.deepcopy(pred)
  if len(pred) > 0:
    rel_truth = np.zeros((1, max(len(pred), len(truth))))
    rel_pred = np.zeros((1, max(len(pred), len(truth))))
    count_t = 0
    for t in truth:
      if t in pred_copy:
        idx = np.where(np.array(pred_copy) == t)[0]
        pred_copy[idx[0]] = 'NAN'
        rel_pred[0, idx[0]]=1
        rel_truth[0, count_t]=1
      count_t+=1
    return ndcg_score(rel_truth, rel_pred, k=K)
  else:
    return 0

In [51]:
def getMostPopularSongs(n, seed_playlist):
  songs_with_freq_list = []
  for song in track_frequency:
    songs_with_freq_list.append((track_frequency[song], song))
  songs_with_freq_list.sort(reverse=True)
  res = [pair[1] for pair in songs_with_freq_list if pair[1] not in seed_playlist]
  return res[:n]

In [ ]:
getArtistsForTracks(getMostPopularSongs(100, ['spotify:track:3a1lNhkSLSkpJE4MSHpDu9']))

In [12]:
validation_data = pd.read_pickle('Validation_dataset.pkl')

In [74]:
def runPopularTrackModel(topKgrid):
  for topK in topKgrid:
    tracks_r_precision = []
    artists_r_precision = []
    ndcg_tracks = []
    ndcg_artists = []
    for idx, row in validation_data.iterrows():
      res = getMostPopularSongs(topK, row['tracks'])
      artists_of_pred = getArtistsForTracks(res)
      tracks_r_precision.append(R_precision(res, row['with_held_tracks']))
      artists_r_precision.append(R_precision(artists_of_pred, row['with_held_artists']))
      ndcg_artists.append(NDCG(row['with_held_artists'], artists_of_pred, topK))
      ndcg_tracks.append(NDCG(row['with_held_tracks'], res, topK))
    print('K=',topK, ' ,RP(Artist)=',sum(artists_r_precision)/280)
    print('K=',topK, ' ,RP(Track)=',sum(tracks_r_precision)/280)
    print('K=',topK, ' ,NDCG(Track)=',sum(ndcg_tracks)/280)
    print('K=',topK, ' ,NDCG(Artist)=',sum(ndcg_artists)/280)
    print('=====================================================')

In [75]:
runPopularTrackModel([10, 20, 30, 40, 50, 60, 70, 100, 200, 300, 400, 500])

K= 10  ,RP(Artist)= 0.03917375283446708
K= 10  ,RP(Track)= 0.0
K= 10  ,NDCG(Track)= 0.0
K= 10  ,NDCG(Artist)= 0.11979168267831135
K= 20  ,RP(Artist)= 0.08119047619047612
K= 20  ,RP(Track)= 0.0
K= 20  ,NDCG(Track)= 0.0
K= 20  ,NDCG(Artist)= 0.15943377412158097
K= 30  ,RP(Artist)= 0.11507369614512462
K= 30  ,RP(Track)= 0.0
K= 30  ,NDCG(Track)= 0.0
K= 30  ,NDCG(Artist)= 0.1863124377856262
K= 40  ,RP(Artist)= 0.13420634920634913
K= 40  ,RP(Track)= 0.0
K= 40  ,NDCG(Track)= 0.0
K= 40  ,NDCG(Artist)= 0.18995615850896774
K= 50  ,RP(Artist)= 0.14615787981859402
K= 50  ,RP(Track)= 0.0
K= 50  ,NDCG(Track)= 0.0
K= 50  ,NDCG(Artist)= 0.18849296609358174
K= 60  ,RP(Artist)= 0.15725340136054416
K= 60  ,RP(Track)= 0.0
K= 60  ,NDCG(Track)= 0.0
K= 60  ,NDCG(Artist)= 0.18940025213123007
K= 70  ,RP(Artist)= 0.17477607709750567
K= 70  ,RP(Track)= 0.0
K= 70  ,NDCG(Track)= 0.0
K= 70  ,NDCG(Artist)= 0.19447242221284639
K= 100  ,RP(Artist)= 0.22205073696145125
K= 100  ,RP(Track)= 0.0
K= 100  ,NDCG(Track)= 0.0


### Recommend Songs of only those Artists corresponding to Seed playlist

In [80]:
def getMostPopularSongsForArtist(seedTracks, artist_name, n, tracks_to_choose_from=None):
  if tracks_to_choose_from is None:
    artist_songs = artist_tracks_dict[artist_name]
    filtered = list(set(artist_songs) - set(seedTracks))
    return random.sample(filtered, min(n, len(filtered)))
  else:
    track_list = tracks_to_choose_from[artist_name]
    filtered = list(set(track_list) - set(seedTracks))
    return random.sample(filtered, min(n, len(filtered)))
  # artist_popular_songs = []
  # for song in artist_songs:
    # artist_popular_songs.append((track_frequency[song], song))
    # artist_popular_songs.sort(reverse=True)
    # filtered = [pair[1] for pair in artist_popular_songs if pair[1] not in seedTracks]
  # return filtered[:n]



In [81]:
def recommendPopularSongsBasedOnArtists(seedTracks, artists, n, tracks_to_choose_from=None):
  total_tracks = 0
  for artist in artists:
    total_tracks += artist_tracks_count_dict[artist]
  results = []
  for artist in artists:
    count = int(math.ceil((n * artist_tracks_count_dict[artist]) / total_tracks))
    results.extend(getMostPopularSongsForArtist(seedTracks, artist, count, tracks_to_choose_from))
  return results[:n]

In [82]:
def runPopularArtistTrackModel(topKgrid):
  for topK in topKgrid:
    tracks_r_precision = []
    artists_r_precision = []
    ndcg_tracks = []
    ndcg_artists = []
    for idx, row in validation_data.iterrows():
      artists = getArtistsForTracks(row['tracks'])
      res = recommendPopularSongsBasedOnArtists(list(set(row['tracks']) - set(row['with_held_tracks'])), artists, topK, None)
      artists_of_pred = getArtistsForTracks(res)
      tracks_r_precision.append(R_precision(res, row['with_held_tracks']))
      artists_r_precision.append(R_precision(artists_of_pred, row['with_held_artists']))
      ndcg_tracks.append(NDCG(row['with_held_tracks'], res, topK))
      ndcg_artists.append(NDCG(row['with_held_artists'], artists_of_pred, topK))
    print('K=',topK, ' ,RP(Artist)=',sum(artists_r_precision)/280)
    print('K=',topK, ' ,RP(Track)=',sum(tracks_r_precision)/280)
    print('K=',topK, ' ,NDCG(Track)=',sum(ndcg_tracks)/280)
    print('K=',topK, ' ,NDCG(Artist)=',sum(ndcg_artists)/280)
    print('=====================================================')

In [83]:
runPopularArtistTrackModel([10, 20, 30, 40, 50, 60, 70, 100, 200, 300, 400, 500])

K= 10  ,RP(Artist)= 0.214933390022676
K= 10  ,RP(Track)= 0.034722222222222196
K= 10  ,NDCG(Track)= 0.09423239558236923
K= 10  ,NDCG(Artist)= 0.456243020584936
K= 20  ,RP(Artist)= 0.3978188775510204
K= 20  ,RP(Track)= 0.06948412698412698
K= 20  ,NDCG(Track)= 0.15390939970024026
K= 20  ,NDCG(Artist)= 0.4854420741046158
K= 30  ,RP(Artist)= 0.533564342403628
K= 30  ,RP(Track)= 0.1062301587301588
K= 30  ,NDCG(Track)= 0.18103731171965867
K= 30  ,NDCG(Artist)= 0.4932499902316047
K= 40  ,RP(Artist)= 0.6069784580498867
K= 40  ,RP(Track)= 0.13496031746031772
K= 40  ,NDCG(Track)= 0.20663531287086545
K= 40  ,NDCG(Artist)= 0.48623864391986615
K= 50  ,RP(Artist)= 0.6770847505668939
K= 50  ,RP(Track)= 0.1750793650793653
K= 50  ,NDCG(Track)= 0.22758796699427034
K= 50  ,NDCG(Artist)= 0.48033384917445027
K= 60  ,RP(Artist)= 0.7127026643990938
K= 60  ,RP(Track)= 0.20634920634920662
K= 60  ,NDCG(Track)= 0.2402515868393371
K= 60  ,NDCG(Artist)= 0.4679260965594029
K= 70  ,RP(Artist)= 0.7400765306122454
K= 7

Higher artist R@K shows users have favourite artists and its common to have majority of playlist composed of their tracks.

In [ ]:
runPopularArtistTrackModel([10, 20, 30, 40, 50, 60, 70])

### Cosine Similarity of Playlist Title Word Embeddings - Find the similarity between query Playlist title and Trainig Playlist Titles(acting as genere)

In [40]:
import zipfile
with zipfile.ZipFile('/content/drive/MyDrive/glove_emb.pb (1).zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [41]:
words_embeddings_dict = defaultdict()
with open('glove_emb.pb', 'rb') as handle:
    words_embeddings_dict = pkl.load(handle)

In [61]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
playlist_names = playlists_data['name']
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [62]:
def getPlaylistTitleEmbedding(name):
  # print(name)
  embeddies = np.zeros((1,300), dtype=float)
  for w in name:
    # print(w)
    if w.lower() in words_embeddings_dict:
      embeddies += np.array(words_embeddings_dict[w.lower()]).reshape((1,300))
  return embeddies

In [63]:
def tracksInPlaylist(p):
  # print(p, len(playlists_data.loc[playlists_data['name']==p,'tracks']))
  return list(playlists_data.loc[playlists_data['name']==p,'tracks'].iloc[0])

In [64]:
def getMostSimilarPlaylist(query_playlist):
  sim_matrix = []
  # best_match = ''
  name1 = [w.lower() for w in query_playlist.strip().split(' ') if not w.lower() in stop_words]
  embedding1 = getPlaylistTitleEmbedding(name1)
  for n2 in playlist_names.to_list():
    # print(n2)
    name2 = [w.lower() for w in n2.strip().split(' ') if not w.lower() in stop_words]
    embedding2 = getPlaylistTitleEmbedding(name2)
    sim = 1 - distance.cosine(embedding2, embedding1)
    sim_matrix.append((sim, n2))
    # print(sim, query_playlist, n2)
    # if(sim > max_sim):
      # max_sim = sim
      # best_match = n2
  return sim_matrix

In [65]:
def getRandomTopNFromBestMatchTitles(query_playlist, n, playlist_cnt, seedTracks):
  sim_matrix = getMostSimilarPlaylist(query_playlist)
  # print(sim_matrix)
  sim_matrix = [pair for pair in sim_matrix if not np.isnan(pair[0])]
  sim_matrix.sort(reverse=True)
  # print(sim_matrix)
  total_sim = sum([pair[0] for pair in sim_matrix[:playlist_cnt]])
  result = []
  for p in sim_matrix[:playlist_cnt]:
    tracks = tracksInPlaylist(p[1])
    weight = int(math.ceil(n * p[0] / total_sim))
    result.extend(random.sample(tracks, min(weight, len(tracks))))
  result = list(set(result) - set(seedTracks))
  return result[:n]


In [66]:
def runWordEmbeddingsModel(topKgrid, playlist_cnt):
  for topK in topKgrid:
    tracks_r_precision = []
    artists_r_precision = []
    ndcg_tracks = []
    ndcg_artists = []
    for idx, row in validation_data.iterrows():
      res = getRandomTopNFromBestMatchTitles(row['name'], topK, playlist_cnt, list(set(row['tracks']) - set(row['with_held_tracks'])))
      artists_of_pred = getArtistsForTracks(res)
      tracks_r_precision.append(R_precision(res, row['with_held_tracks']))
      artists_r_precision.append(R_precision(artists_of_pred, row['with_held_artists']))
      ndcg_tracks.append(NDCG(row['with_held_tracks'], res, topK))
      ndcg_artists.append(NDCG(row['with_held_artists'], artists_of_pred, topK))
    print('K=',topK, ' ,RP(Artist)=',sum(artists_r_precision)/280)
    print('K=',topK, ' ,RP(Track)=',sum(tracks_r_precision)/280)
    # print(ndcg_tracks)
    print('K=',topK, ' ,NDCG(Track)=',sum(ndcg_tracks)/280)
    print('K=',topK, ' ,NDCG(Artist)=',sum(ndcg_artists)/280)
    print('=====================================================')

In [67]:
warnings.filterwarnings('ignore')

In [68]:
runWordEmbeddingsModel([10, 20, 30, 40, 50, 60, 70, 100, 200, 300, 400, 500], 1)

K= 10  ,RP(Artist)= 0.04570578231292513
K= 10  ,RP(Track)= 0.01833333333333333
K= 10  ,NDCG(Track)= 0.06291052150535761
K= 10  ,NDCG(Artist)= 0.13155640410790367
K= 20  ,RP(Artist)= 0.07417800453514738
K= 20  ,RP(Track)= 0.03087301587301585
K= 20  ,NDCG(Track)= 0.0728763323061193
K= 20  ,NDCG(Artist)= 0.15712506170174034
K= 30  ,RP(Artist)= 0.0846301020408163
K= 30  ,RP(Track)= 0.039603174603174585
K= 30  ,NDCG(Track)= 0.07053772711363447
K= 30  ,NDCG(Artist)= 0.1521008774652166
K= 40  ,RP(Artist)= 0.09463293650793653
K= 40  ,RP(Track)= 0.04781746031746028
K= 40  ,NDCG(Track)= 0.07784473289980276
K= 40  ,NDCG(Artist)= 0.15820276874378117
K= 50  ,RP(Artist)= 0.09875425170068028
K= 50  ,RP(Track)= 0.049603174603174566
K= 50  ,NDCG(Track)= 0.07884029391572345
K= 50  ,NDCG(Artist)= 0.1616663920807967
K= 60  ,RP(Artist)= 0.09926445578231294
K= 60  ,RP(Track)= 0.049603174603174566
K= 60  ,NDCG(Track)= 0.07938248988779682
K= 60  ,NDCG(Artist)= 0.156823019884791
K= 70  ,RP(Artist)= 0.099264455

## with top 4 matches

In [69]:
runWordEmbeddingsModel([10, 20, 30, 40, 50, 60, 70, 100, 200, 300, 400, 500], 4)

K= 10  ,RP(Artist)= 0.03327806122448978
K= 10  ,RP(Track)= 0.010039682539682543
K= 10  ,NDCG(Track)= 0.03167864019281368
K= 10  ,NDCG(Artist)= 0.08836972474831191
K= 20  ,RP(Artist)= 0.067906746031746
K= 20  ,RP(Track)= 0.019761904761904755
K= 20  ,NDCG(Track)= 0.05481885881760217
K= 20  ,NDCG(Artist)= 0.15196403036147452
K= 30  ,RP(Artist)= 0.09063066893424034
K= 30  ,RP(Track)= 0.029166666666666653
K= 30  ,NDCG(Track)= 0.06626443129997632
K= 30  ,NDCG(Artist)= 0.1736726433455455
K= 40  ,RP(Artist)= 0.10067176870748296
K= 40  ,RP(Track)= 0.03555555555555554
K= 40  ,NDCG(Track)= 0.060149770123431444
K= 40  ,NDCG(Artist)= 0.1664665951968735
K= 50  ,RP(Artist)= 0.10910856009070291
K= 50  ,RP(Track)= 0.04115079365079363
K= 50  ,NDCG(Track)= 0.06603014358305857
K= 50  ,NDCG(Artist)= 0.1603242843007847
K= 60  ,RP(Artist)= 0.13150935374149672
K= 60  ,RP(Track)= 0.04869047619047616
K= 60  ,NDCG(Track)= 0.0652625068432118
K= 60  ,NDCG(Artist)= 0.17732975560631006
K= 70  ,RP(Artist)= 0.13413832

## with top 3 matches

In [70]:
runWordEmbeddingsModel([10, 20, 30, 40, 50, 60, 70, 100, 200, 300, 400, 500], 3)

K= 10  ,RP(Artist)= 0.03278061224489795
K= 10  ,RP(Track)= 0.009682539682539685
K= 10  ,NDCG(Track)= 0.029695287777445664
K= 10  ,NDCG(Artist)= 0.08836530140666722
K= 20  ,RP(Artist)= 0.064672619047619
K= 20  ,RP(Track)= 0.02265873015873015
K= 20  ,NDCG(Track)= 0.05986674406359923
K= 20  ,NDCG(Artist)= 0.15485468814255035
K= 30  ,RP(Artist)= 0.0760926870748299
K= 30  ,RP(Track)= 0.031626984126984094
K= 30  ,NDCG(Track)= 0.06747680713699819
K= 30  ,NDCG(Artist)= 0.14125622683861394
K= 40  ,RP(Artist)= 0.10361394557823127
K= 40  ,RP(Track)= 0.041428571428571405
K= 40  ,NDCG(Track)= 0.06616274666940886
K= 40  ,NDCG(Artist)= 0.16959972096454193
K= 50  ,RP(Artist)= 0.11286989795918366
K= 50  ,RP(Track)= 0.04936507936507931
K= 50  ,NDCG(Track)= 0.07769312701551474
K= 50  ,NDCG(Artist)= 0.16636481360447145
K= 60  ,RP(Artist)= 0.12200963718820866
K= 60  ,RP(Track)= 0.04960317460317455
K= 60  ,NDCG(Track)= 0.07208724422211621
K= 60  ,NDCG(Artist)= 0.16666515007909422
K= 70  ,RP(Artist)= 0.13111

## with top 2 matches

In [ ]:
runWordEmbeddingsModel([10, 20, 30, 40, 50, 60, 70, 100, 200, 300, 400, 500], 2)

In [72]:
runWordEmbeddingsModel([10, 20, 30, 40, 50, 60, 70, 100, 200, 300, 400, 500], 10)

K= 10  ,RP(Artist)= 0.039606009070294754
K= 10  ,RP(Track)= 0.008412698412698415
K= 10  ,NDCG(Track)= 0.02849204429717998
K= 10  ,NDCG(Artist)= 0.11229172059870464
K= 20  ,RP(Artist)= 0.07356150793650791
K= 20  ,RP(Track)= 0.017658730158730155
K= 20  ,NDCG(Track)= 0.04973877960059961
K= 20  ,NDCG(Artist)= 0.15471609921470675
K= 30  ,RP(Artist)= 0.08196003401360541
K= 30  ,RP(Track)= 0.023730158730158716
K= 30  ,NDCG(Track)= 0.05235703656123499
K= 30  ,NDCG(Artist)= 0.15640455229870046
K= 40  ,RP(Artist)= 0.11402210884353736
K= 40  ,RP(Track)= 0.03599206349206345
K= 40  ,NDCG(Track)= 0.06992129305068595
K= 40  ,NDCG(Artist)= 0.1807183718138687
K= 50  ,RP(Artist)= 0.11852749433106573
K= 50  ,RP(Track)= 0.03960317460317457
K= 50  ,NDCG(Track)= 0.06465388097440485
K= 50  ,NDCG(Artist)= 0.1656013982181892
K= 60  ,RP(Artist)= 0.13784580498866217
K= 60  ,RP(Track)= 0.04646825396825392
K= 60  ,NDCG(Track)= 0.07228794257349476
K= 60  ,NDCG(Artist)= 0.17743546622467388
K= 70  ,RP(Artist)= 0.1514

##Combine Word Embeddings with Artist popularity (No need to read)

In [ ]:
def getTrackDictForArtistWithinPlayists(sim_matrix):
  all_tracks= []
  for p in sim_matrix:
    all_tracks.extend(tracksInPlaylist(p[1]))
  artist_track_dict = defaultdict(list)
  for track in all_tracks:
    artist_track_dict[track_artist_mapping[track]].append(track)
  return artist_track_dict

In [ ]:
def runPopularArtistPlusWordEmbeddingModel(topKgrid):
  for topK in topKgrid:
    tracks_r_precision = []
    artists_r_precision = []
    ndcg_tracks = []
    for idx, row in validation_data.iterrows():
      sim_matrix = getMostSimilarPlaylist(row['name'])
      sim_matrix = [pair for pair in sim_matrix if not np.isnan(pair[0])]
      sim_matrix.sort(reverse=True)
      tracks_to_choose_from = getTrackDictForArtistWithinPlayists(sim_matrix[:20])
      res = recommendPopularSongsBasedOnArtists(row['tracks'], list(tracks_to_choose_from.keys()), topK, tracks_to_choose_from)
      artists_of_pred = getArtistsForTracks(res)
      tracks_r_precision.append(R_precision(res, row['with_held_tracks']))
      artists_r_precision.append(R_precision(artists_of_pred, row['with_held_artists']))
      ndcg_tracks.append(NDCG(row['with_held_tracks'], res, topK))
    print('K=',topK, ' ,RP(Artist)=',sum(artists_r_precision)/280)
    print('K=',topK, ' ,RP(Track)=',sum(tracks_r_precision)/280)
    print('K=',topK, ' ,NDCG(Track)=',sum(ndcg_tracks)/280)
    print('=====================================================')

In [ ]:
runPopularArtistPlusWordEmbeddingModel([10, 20, 30, 40, 50, 60, 70, 100])

K= 10  ,RP(Artist)= 0.06216128117913829
K= 10  ,RP(Track)= 0.01301587301587302
K= 10  ,NDCG(Track)= 0.048315354148027116
K= 20  ,RP(Artist)= 0.09726899092970523
K= 20  ,RP(Track)= 0.025198412698412685
K= 20  ,NDCG(Track)= 0.058512025778202456
K= 30  ,RP(Artist)= 0.12450538548752842
K= 30  ,RP(Track)= 0.029087301587301568
K= 30  ,NDCG(Track)= 0.06067194781868334
K= 40  ,RP(Artist)= 0.14595096371882102
K= 40  ,RP(Track)= 0.03734126984126981
K= 40  ,NDCG(Track)= 0.07399425398276945
K= 50  ,RP(Artist)= 0.16419217687074844
K= 50  ,RP(Track)= 0.04345238095238091
K= 50  ,NDCG(Track)= 0.07631631634538912
K= 60  ,RP(Artist)= 0.18302295918367362
K= 60  ,RP(Track)= 0.04638888888888884
K= 60  ,NDCG(Track)= 0.07543499597435674
K= 70  ,RP(Artist)= 0.19517006802721104
K= 70  ,RP(Track)= 0.048809523809523754
K= 70  ,NDCG(Track)= 0.07278133910441542
K= 100  ,RP(Artist)= 0.23253259637188245
K= 100  ,RP(Track)= 0.06246031746031743
K= 100  ,NDCG(Track)= 0.08485086070108266


##With 20 similar playlists

In [ ]:
runPopularArtistPlusWordEmbeddingModel([10, 20, 30, 40, 50, 60, 70])

Word embeddings are not giving ggod results not even with same artists as of seed list. Which means not all songs are related to the title of the playlist. The songs are random and do not describe the overall genere of songs. 

In [ ]:
testing_dataset = pd.read_pickle('Test_dataset.pkl')

In [ ]:
sum(testing_dataset['num_tracks'])

2463